<a href="https://colab.research.google.com/github/KSY1526/SSUDA_LOTTEs/blob/main/DUCK/LPOINT_BIG_COMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# -----------------------무시

품목별 소비자물가지수

https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1J20005&vw_cd=MT_ZTITLE&list_id=P2_6&scrId=&seqNo=&lang_mode=ko&obj_var_id=&itm_id=&conn_path=E1

 기존 고객을 유지하는 것이 새로운 고객을 확보하는 것이 훨씬 더 경제적
 실제로 신규 고객의 
획득 비용은 기존 고객의 유지비용 대비 5~6배에 달하는 것으로 알려져 있다

https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=gojump0713&logNo=140132035618

In [1]:
!head -n 3 /proc/meminfo

MemTotal:       26692024 kB
MemFree:        24025928 kB
MemAvailable:   25741868 kB


In [2]:
%%capture
!pip install bertopic

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
path = "/content/drive/MyDrive/AIFactory/Data/LPOINT_BIG_COMP/LPOINT_BIG_COMP_PROCESSED_CUST/"

# COPU = pd.read_csv(path + "LPOINT_BIG_COMP_COPU_CUST.csv")
# LPAY = pd.read_csv(path + "LPOINT_BIG_COMP_LPAY_CUST.csv")
# PDDE = pd.read_csv(path + "LPOINT_BIG_COMP_PDDE_CUST.csv")
# ind = pd.read_csv(path + "품목별_소비자물가지수_품목성질별_2020100__20220713103547.csv", encoding = 'cp949')

In [4]:

DEMO = pd.read_csv(path + "LPOINT_BIG_COMP_01_DEMO.csv")
PDDE = pd.read_csv(path + "LPOINT_BIG_COMP_02_PDDE.csv")
COP_U = pd.read_csv(path + "LPOINT_BIG_COMP_03_COP_U.csv")
PD_CLAC = pd.read_csv(path + "LPOINT_BIG_COMP_04_PD_CLAC.csv")
BR = pd.read_csv(path + "LPOINT_BIG_COMP_05_BR.csv")
LPAY = pd.read_csv(path + "LPOINT_BIG_COMP_06_LPAY.csv")

ind = pd.read_csv(path + "품목별_소비자물가지수_품목성질별_2020100__20220713103547.csv", encoding = 'cp949')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
path = "/content/drive/MyDrive/AIFactory/Data/LPOINT_BIG_COMP/LPOINT_BIG_COMP_PROCESSED_CUST/"

COPU = pd.read_csv(path + "LPOINT_BIG_COMP_COPU_CUST.csv")
LPAY = pd.read_csv(path + "LPOINT_BIG_COMP_LPAY_CUST.csv")
PDDE = pd.read_csv(path + "LPOINT_BIG_COMP_PDDE_CUST.csv")

In [6]:
COP_U.head(2)

,cust,rct_no,cop_c,br_c,chnl_dv,de_dt,vst_dt,de_hr,buy_am
0,M839993508,21102612B015763935,B01,B010012,1,20211026,20211026,12,60000
1,M839993508,21110610B014219744,B01,B010012,1,20211106,20211106,10,17100


In [7]:
LPAY.head(2)

,Unnamed: 0,cust,ma_fem_dv,ages,zon_hlv,de_dt,de_hr,chnl_dv,cop_c,buy_am
0,0,M629656521,남성,40대,Z04,20210803,21,1,A03,10900
1,1,M216016456,여성,40대,Z04,20210803,13,2,L01,6860


In [8]:
def trans(ch):
    return ch[:8] + ch[11:]

In [9]:
COP_U.rct_no = list(map(trans, COP_U.rct_no))

In [10]:
len(COP_U.rct_no)

248304

In [ ]:
# len(LPAY.rct_no)

In [ ]:
len(set(COP_U.rct_no) & set(LPAY.rct_no))

In [ ]:
# pd.DataFrame(ind.품목별.unique()).to_csv('지수.csv', index = False, encoding = 'ms949')
# pd.DataFrame(PDDE.pd_nm.unique()).to_csv("PDDE.csv", index = False, encoding = 'ms949')

In [ ]:
PDDE['lpay'] = 0
COPU['lpay'] = 0 
LPAY['lpay'] = 1 

In [ ]:
col = ['cust', 'ma_fem_dv', 'ages', 'zon_hlv', 'de_dt', 'de_hr', 'chnl_dv', 'cop_c', 'buy_am', 'lpay']

In [18]:
path = "/content/drive/MyDrive/AIFactory/Data/LPOINT_BIG_COMP/LPOINT_BIG_COMP_PROCESSED_CUST/"


# LPAY_COPU_PDDE = pd.concat([LPAY[col], COPU[col], PDDE[col]])
# LPAY_COPU_PDDE['prop'] = 0

# for id in tqdm(LPAY_COPU_PDDE.cust.unique()):
#     temp = LPAY_COPU_PDDE.loc[LPAY_COPU_PDDE.cust == id]
#     LPAY_COPU_PDDE.loc[LPAY_COPU_PDDE.cust == id, 'prop'] = temp.lpay.mean()
# LPAY_COPU_PDDE.to_csv(path + "LPAY_COPU_PDDE.csv", index = False) 
LPAY_COPU_PDDE = pd.read_csv(path + "LPAY_COPU_PDDE.csv")

# --------------------- 여기서부터

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset


from sklearn.preprocessing import LabelEncoder

In [2]:

path = "/content/drive/MyDrive/AIFactory/Data/LPOINT_BIG_COMP/"

DEMO = pd.read_csv(path + "LPOINT_BIG_COMP_01_DEMO.csv")
PDDE = pd.read_csv(path + "LPOINT_BIG_COMP_02_PDDE.csv")
COP_U = pd.read_csv(path + "LPOINT_BIG_COMP_03_COP_U.csv")
PD_CLAC = pd.read_csv(path + "LPOINT_BIG_COMP_04_PD_CLAC.csv")
BR = pd.read_csv(path + "LPOINT_BIG_COMP_05_BR.csv")
LPAY = pd.read_csv(path + "LPOINT_BIG_COMP_06_LPAY.csv")

# ind = pd.read_csv(path + "품목별_소비자물가지수_품목성질별_2020100__20220713103547.csv", encoding = 'cp949')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
PDDE = PDDE.sort_values('cust')
PDDE.buy_ct = PDDE.buy_ct.astype(np.int8)

In [4]:
n = 5
cnt = 0
tmp_array = np.array([])
df = pd.DataFrame()
for i in range(1):
    if i == 8:
        tem = pd.pivot_table(PDDE[500000 * i :], index = ['cust', 'rct_no'], columns = 'pd_c', values = 'buy_ct', aggfunc = 'sum' )
    else:
        tem = pd.pivot_table(PDDE[500000 * i: 500000*(i+1)], index = ['cust', 'rct_no'], columns = 'pd_c', values = 'buy_ct', aggfunc = 'sum')
    tem.fillna(0, inplace = True)
    tem = tem.astype(np.int8)
    tem.reset_index(['rct_no'], drop = True, inplace = True)

    for id in tqdm(tem.index.unique()):
        if np.sum(tem.index == id) < n:
            tem.drop(id, axis = 0,inplace = True)    

    df = pd.concat([df, tem]).fillna(0)
    df = df.astype(np.int8)

for id in tqdm(df.index.unique()):
    tmp = df.loc[df.index == id]
    t1 = tmp.shape[0] // n
    t2 = tmp.shape[0] % n
    for j in range(t1):
        tmp_array = np.append(tmp_array, np.repeat(cnt, n), axis = 0)
        cnt += 1
    tmp_array = np.append(tmp_array, np.repeat(-1, t2))

df['id'] = tmp_array
df = pd.merge(df, DEMO, on = 'cust')

df.drop(np.where(df.id == -1)[0], axis = 0, inplace = True)
df.drop(['cust'], axis = 1, inplace = True)

df.reset_index(drop = True, inplace = True)

le = LabelEncoder()
le.fit(df['ma_fem_dv'])
df['ma_fem_dv'] = le.transform(df['ma_fem_dv'])

le.fit(df['ages'])
df['ages'] = le.transform(df['ages'])

le.fit(df['zon_hlv'])
df['zon_hlv'] = le.transform(df['zon_hlv'])

# df.to_csv(path + "df.csv", index = False)
# df = pd.read_csv(path + "df.csv")

100%|██████████| 2559/2559 [00:25<00:00, 98.64it/s]


In [5]:
df.head()

,PD0001,PD0002,PD0003,PD0004,PD0005,PD0006,PD0007,PD0008,PD0009,PD0010,...,PD1928,PD1929,PD1930,PD1931,PD1932,PD1933,id,ma_fem_dv,ages,zon_hlv
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,1,2,6
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,1,2,6
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,1,2,6
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,1,2,6
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,1,2,6


In [6]:
class CustomDataset(Dataset):
    def __init__(self, df, id, train_mode = True, transforms = None, n = None):
        self.transforms = transforms
        self.train_mode = train_mode
        self.df = df
        self.id = id
        self.n = n

    def __len__(self):
        return len(self.id)

    def __getitem__(self, idx):
        id = self.id[idx]
        input = self.df.loc[df.id == id].drop(['id'], axis = 1)

        if self.train_mode:
            output = input.iloc[:,:-4]
            return torch.tensor(np.array(input)), torch.tensor(np.array(output))

        else:
            return torch.tensor(np.array(input))

In [7]:
class Encoder(nn.Module):
    def __init__(self, seq_len, n_features, embedding_dim=64):
        super(Encoder, self).__init__()
        self.seq_len, self.n_features = seq_len, n_features
        self.embedding_dim, self.hidden_dim = (
            embedding_dim, 2 * embedding_dim
        )
        self.rnn1 = nn.LSTM(
          input_size=n_features,
          hidden_size=self.hidden_dim,
          num_layers=1,
          batch_first=True
        )
        self.rnn2 = nn.LSTM(
          input_size=self.hidden_dim,
          hidden_size=embedding_dim,
          num_layers=1,
          batch_first=True
        )
    def forward(self, x):
        x, (_, _) = self.rnn1(x)
        x, (hidden_n, _) = self.rnn2(x)
        return  x[:,-1,:]

In [8]:
class TimeDistributed(nn.Module):
    def __init__(self, module, batch_first=False):
        super(TimeDistributed, self).__init__()
        self.module = module
        self.batch_first = batch_first

    def forward(self, x):
        if len(x.size()) <= 2:
            return self.module(x)
        # Squash samples and timesteps into a single axis
        x_reshape = x.contiguous().view(-1, x.size(-1))  # (samples * timesteps, input_size)
        y = self.module(x_reshape)
        # We have to reshape Y
        if self.batch_first:
            y = y.contiguous().view(x.size(0), -1, y.size(-1))  # (samples, timesteps, output_size)
        else:
            y = y.view(-1, x.size(1), y.size(-1))  # (timesteps, samples, output_size)
        return y


In [9]:
class Decoder(nn.Module):
    def __init__(self, seq_len, input_dim=64, n_features=1):
        super(Decoder, self).__init__()
        self.seq_len, self.input_dim = seq_len, input_dim
        self.hidden_dim, self.n_features = 2 * input_dim, n_features
        self.rnn1 = nn.LSTM(
          input_size=input_dim,
          hidden_size=input_dim,
          num_layers=1,
          batch_first=True
        )
        self.rnn2 = nn.LSTM(
          input_size=input_dim,
          hidden_size=self.hidden_dim,
          num_layers=1,
          batch_first=True
        )
        self.output_layer = torch.nn.Linear(self.hidden_dim, n_features)
        self.timedist = TimeDistributed(self.output_layer)
        
    def forward(self, x):
        x=x.reshape(-1,1,self.input_dim).repeat(1,self.seq_len,1)       
        x, (hidden_n, cell_n) = self.rnn1(x)
        x, (hidden_n, cell_n) = self.rnn2(x)
        return self.timedist(x)

In [10]:
class RecurrentAutoencoder(nn.Module):
    def __init__(self, seq_len, n_features, embedding_dim=64):
        super(RecurrentAutoencoder, self).__init__()
        self.encoder = Encoder(seq_len, n_features, embedding_dim)#.to(device)
        self.decoder = Decoder(seq_len, embedding_dim, n_features - 4)#.to(device)
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [11]:
df

,PD0001,PD0002,PD0003,PD0004,PD0005,PD0006,PD0007,PD0008,PD0009,PD0010,...,PD1928,PD1929,PD1930,PD1931,PD1932,PD1933,id,ma_fem_dv,ages,zon_hlv
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,1,2,6
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,1,2,6
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,1,2,6
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,1,2,6
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,1,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137575,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,27515.0,1,1,14
137576,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,27515.0,1,1,14
137577,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,27515.0,1,1,14
137578,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,27515.0,1,1,14


In [12]:
#5 X 1782  시계열 데이터를 1 X 128 형태로 변환
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = RecurrentAutoencoder(5, 1782, 128)
model = model.to(device)

In [13]:
from sklearn.model_selection import train_test_split

train_id, valid_id = train_test_split(df.id.unique(), test_size = 0.2, random_state = 42)


In [14]:
train_dataset = CustomDataset(df, train_id, train_mode = True, transforms = None, n = 5)
train_loader = DataLoader(train_dataset, batch_size = 128, shuffle=False, num_workers=0)

valid_dataset = CustomDataset(df, valid_id, train_mode = True, transforms = None, n = 5)
valid_loader = DataLoader(train_dataset, batch_size = 128, shuffle=False, num_workers=0)

In [15]:
def validation(model, vali_loader, criterion, device):
    model.eval() # Evaluation
    vali_loss = []
    with torch.no_grad():
        for idx, batch in enumerate(train_loader):
            x, y = batch[0].float().to(device), batch[1].float().to(device)

            pred = model(x)
            loss = criterion(pred, y)
            
            vali_loss.append(loss.item())

    vali_mae_loss = np.mean(vali_loss)
    return vali_mae_loss

In [16]:
    # def train(model, optimizer, train_loader, vali_loader, scheduler, device):

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    # Loss Function
    criterion = nn.L1Loss(reduction='sum').to(device)
    best_mae = 9999
    
    for epoch in range(50):
        model.train()
        train_loss = []
        for idx, batch in enumerate(train_loader):
            x, y = batch[0].float().to(device), batch[1].float().to(device)
            optimizer.zero_grad()
            # Data -> Model -> Output
            pred = model(x)
            # logit = (483-0) * logit + 0       # sigmoid 함수 사용했을 때
            # Calc loss
            loss = criterion(pred, y)

            # backpropagation
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())
                        
        # Evaluation Validation set
        vali_mae = validation(model, valid_loader, criterion, device)
        
        print(f'Epoch [{epoch}] Train MAE : [{np.mean(train_loss):.5f}] Validation MAE : [{vali_mae:.5f}]\n')
        


KeyboardInterrupt: ignored

#_--------------------------------

In [13]:
col = ['cust', 'ma_fem_dv', 'ages', 'prop']

In [ ]:
LPAY_COPU_PDDE = LPAY_COPU_PDDE[col].drop_duplicates()

In [ ]:
LPAY_COPU_PDDE

In [ ]:
tem.reset_index(inplace = True)

In [ ]:
tem

In [ ]:
df = pd.merge(LPAY_COPU_PDDE, tem, on = 'cust')
df.head()

,cust,ma_fem_dv,ages,prop,PD00011,PD00012,PD00021,PD00022,PD00031,PD00032,...,PD19291,PD19292,PD19301,PD19302,PD19311,PD19312,PD19321,PD19322,PD19331,PD19332
0,M629656521,남성,40대,0.007042,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M216016456,여성,40대,0.265957,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M205142844,여성,60대,0.187747,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M737010483,남성,40대,0.076149,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M707775545,남성,30대,0.020080,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
docs = []
for i in tqdm(range(df.shape[0])):
    do = []
    for j in range(1, df.shape[1]):
        if j < 3:
            do.append(df.iloc[i,j])
        elif j == 3:
            for k in range(int((df.iloc[i, j] * 100) //1)):
                do.append('lpay')
        else:
            for k in range(int(df.iloc[i, j])):
                do.append(df.columns[j])
        # random.shuffle(do)        # 이게 오래걸림
    docs.append(' '.join(do))

In [ ]:
PDDE

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,buy
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000.0,1,PD02902
1,M646853852,A01000002265,1,A01,A010025,PD1369,20210101,10,79700.0,1,PD13691
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1,PD02902
3,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1,PD02902
4,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000.0,1,PD02902
...,...,...,...,...,...,...,...,...,...,...,...
4381738,M533286446,E06052119978,2,A06,NaN,PD0507,20211231,23,50000.0,1,PD05071
4381739,M533286446,E06052119978,2,A06,NaN,PD0507,20211231,23,50000.0,1,PD05071
4381740,M533286446,E06052119978,2,A06,NaN,PD0507,20211231,23,50000.0,1,PD05071
4381741,M533286446,E06052119978,2,A06,NaN,PD0507,20211231,23,50000.0,1,PD05071


In [ ]:
df.iloc[i]

cust         M811831696
ma_fem_dv            남성
ages                40대
prop           0.136819
PD00011             0.0
                ...    
PD19312             0.0
PD19321             0.0
PD19322             0.0
PD19331             0.0
PD19332             0.0
Name: 319, Length: 3824, dtype: object

In [ ]:
' '.join(do)

'남성 40대 lpay lpay lpay lpay lpay lpay lpay lpay lpay lpay lpay lpay lpay PD00952 PD01002 PD01002 PD01021 PD01151 PD01152 PD01152 PD01152 PD01152 PD01182 PD01542 PD01771 PD01862 PD02042 PD02042 PD02092 PD02092 PD02092 PD02092 PD02092 PD02172 PD02181 PD02182 PD02182 PD02232 PD02282 PD02301 PD02301 PD02321 PD02321 PD02321 PD02321 PD02321 PD02321 PD02321 PD02321 PD02322 PD02322 PD02322 PD02371 PD02421 PD02421 PD02422 PD02821 PD02821 PD02821 PD02822 PD02822 PD02982 PD03182 PD03182 PD03182 PD03192 PD03192 PD03251 PD03252 PD03252 PD03252 PD03252 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03282 PD03302 PD03302 PD03302 PD03302 PD03302 PD03302

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(language="multilingual", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(docs)

Batches:   0%|          | 0/365 [00:00<?, ?it/s]

2022-07-20 03:12:08,021 - BERTopic - Transformed documents to Embeddings
2022-07-20 03:12:40,380 - BERTopic - Reduced dimensionality
2022-07-20 03:12:55,881 - BERTopic - Clustered reduced embeddings


In [ ]:
freq = topic_model.get_topic_info(); freq

,Topic,Count,Name
0,-1,3477,-1_lpay_pd11731_pd04801_pd02321
1,0,771,0_pd13311_pd02821_pd02321_pd14841
2,1,686,1_pd02322_pd14841_pd04801_pd14842
3,2,279,2_pd13301_pd11731_pd03811_pd02281
4,3,259,3_pd03191_pd02322_30대_pd14841
...,...,...,...
166,165,11,165_pd09581_pd02882_pd09002_pd02881
167,166,10,166_pd11731_pd15042_pd11811_pd02211
168,167,10,167_pd11812_pd16861_pd09901_pd16862
169,168,10,168_pd10202_pd10201_pd11541_pd14841


In [ ]:
topic_model.get_topic(0)  # Select the most frequent topic

[('pd13311', 0.007768531689429481),
 ('pd02821', 0.0065255809915866735),
 ('pd02321', 0.006128607206879618),
 ('pd14841', 0.005954227724696965),
 ('pd04131', 0.005758621796182247),
 ('pd11811', 0.005713665845952688),
 ('pd02322', 0.005662516596981718),
 ('pd13301', 0.0054819762782706795),
 ('pd03811', 0.005462288362838171),
 ('pd11522', 0.005418319761825938)]

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_distribution(probs[200], min_probability=0.015)

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=50)

In [ ]:
topic_model.visualize_barchart(top_n_topics=5)

In [ ]:
topic_model.visualize_heatmap(n_clusters=2, width=1000, height=1000)

In [ ]:
topic_model.visualize_term_rank()